# Growth Data Scientist – Test technique

Travail réalisé par Hugo Palmer, dans le cadre d'une candidature chez Deezer.

Mise en forme dans le Jupyter Notebook, code: Python 3.5

In [1]:
#Importation des librairies classiques
import numpy as np
import pandas as pd

## Profil d'écoutes

### Génération de fichiers d'écoutes aléatoires

Comme les fichiers de données ne sont pas fournis, je crée une fonction qui les génère automatiquement.

In [2]:
def generate_file_day(nb_users=20, nb_artists=10, nb_lines=1000):
    countries = ['FR', 'UK']
    id_users = np.random.randint(nb_users, size=(nb_lines))
    #we affect to each user a country. We assume they do not change of country the same day
    country_of_user = np.random.choice(countries, size=(nb_users), p=[0.7, 0.3])
    id_countries = [country_of_user[id_users[i]] for i in range(nb_lines)]
    
    id_tracks = np.random.randint(nb_artists*10, size=(nb_lines))
    #we assume that the artist of track_id is: track_id%nb_artists
    id_artists = id_tracks % nb_artists
    
    # creates the dictionnary to prepare the creation of the dataframe
    d = {'user_id':id_users, 'country': id_countries, 'artist_id':id_artists, 'track_id':id_tracks}
    data = pd.DataFrame(data=d)
    
    return data

In [3]:
data1 = generate_file_day()
data2 = generate_file_day()
data3 = generate_file_day()

In [4]:
data1.columns

Index(['artist_id', 'country', 'track_id', 'user_id'], dtype='object')

### Définition d'un profil d'écoutes par utilisateur

Nous supposons que les utilisateurs ne changent pas d'un jour à l'autre (cependant, cela ne serait pas très compliqué à prendre en compte si besoin). Nous définissons une table :
- l'index est le user_id
- l'unique colonne est une liste:
        - chaque élément de cette liste est un couple d'integer: 
            * le track_id
            * le nombre de fois que ce track a été écouté dans l'histoire d'écoute de cet user_id
Ainsi, chaque fois que nous devons mettre à jour le profil d'utilisateur avec un fichier de logs d'une journée, pour chaque ligne du fichier log, on accède à la ligne correspondante dans le profil, on ajoute le track_id s'il n'est pas dans la liste de morceaux déjà écoutés, ou on incrémente le nombre de fois que ce morceau a été écouté s'il était déjà présent

In [5]:
users_list = np.array(range(20))
list_dicts = [{} for i in range(20)]

In [6]:
#takes as input a data frame line with the following columns: 'artist_id', 'country', 'track_id', 'user_id]
def add_track_to_user(df_line, dict_user):
    #If the track_id is not in the user profile, we add it 
    if df_line.track_id not in dict_user.keys():
        dict_user[df_line.track_id] = 1 #initialization to 1: the track has been heard once by the user
    else:
        dict_user[df_line.track_id] +=1
    return dict_user

We read the file line per line (here it is a pandas data frame, but we do as if if was a line per line reader of a csv file)

In [7]:
def add_log_to_profiles(data_day):
    for index, row in data_day.iterrows():
        add_track_to_user(row, list_dicts[row.user_id])

In [8]:
add_log_to_profiles(data1)

For example, the history of listening of the user_id=0 is:

In [9]:
print(list_dicts[0])

{1: 1, 67: 1, 4: 1, 97: 1, 72: 1, 9: 1, 10: 1, 14: 1, 79: 2, 16: 1, 81: 1, 18: 1, 83: 1, 85: 1, 87: 1, 25: 1, 90: 1, 27: 1, 29: 1, 30: 1, 96: 1, 33: 1, 99: 1, 36: 3, 39: 2, 40: 1, 78: 1, 45: 1, 46: 1, 54: 1, 57: 1, 59: 2, 60: 1, 61: 1}


We are able to easily update the logs when we get more data from new days:

In [10]:
add_log_to_profiles(data2)
add_log_to_profiles(data3)

The history of listening of the user_id=0 is now:

In [11]:
print(list_dicts[0])

{0: 2, 1: 1, 4: 1, 9: 2, 10: 3, 14: 4, 16: 1, 17: 1, 18: 1, 20: 1, 23: 1, 24: 1, 25: 3, 26: 1, 27: 2, 28: 2, 29: 2, 30: 2, 31: 1, 33: 1, 34: 3, 35: 1, 36: 4, 38: 1, 39: 2, 40: 1, 41: 1, 42: 1, 43: 3, 44: 4, 45: 3, 46: 2, 47: 1, 50: 2, 51: 2, 53: 1, 54: 4, 55: 1, 56: 3, 57: 4, 58: 3, 59: 3, 60: 2, 61: 2, 63: 2, 65: 2, 67: 1, 69: 1, 70: 1, 71: 1, 72: 3, 73: 2, 74: 4, 75: 2, 76: 3, 77: 1, 78: 1, 79: 2, 81: 1, 82: 1, 83: 2, 85: 3, 87: 2, 88: 1, 90: 1, 91: 1, 93: 2, 95: 1, 96: 2, 97: 3, 99: 1}


## Proximité d'utilisateurs

### a - Définition d'une mesure de similarité

Nous avons défini dans la partie précédente un profil d'écoutes pour les utilisateurs. L'étpe suivante est de le normaliser, pour obtenir une distribution de probabilités d'écoutes de chaque morceau par chaque utilisateur.

Pour un utilisateur i, on définit le vecteur d'écoutes normalisé X_i, de taille [nombre de tracks dans le catalogue], tel que pour un track k, X_i[k] est le nombre d'écoutes du morceau k divisé par le nombre total de morceaux écoutés. Pour deux utilisateurs i et j, on peut alors calculer une similarité comme: 

s(i,j) = \sum_{k} |X_i[k]- X_j[k]|

Ainsi, deux utilisateurs écoutat souvant les mêmes morceaux seront considérés comme 'proches'.

### b - Taille, propriétés et stockage de la matrice

Cette matrice doit avoir les propriétés suivantes:
    - symétrique
    - à diagonale nulle
    - On choisit de la normaliser sur [0,1] (c'est un parti-pris)
    
Par ailleurs, en la stockant sous forme de matrice, elle aurait 16M*16M= 256 000 millards d'éléments, ce qui est beaucoup trop gros pour être chargé en mémoire vive.

On pourrait donc la stocker dans une base de données:
    - index1 = utilisateur i
    - index2 = utilisateur i
    - value = valeur de la similarité M(i,j)

Comme la matrice est symétrique, on peut ne stocker que la première moitié des données; par exemple, ne garder une ligne que si user_id(i)<=user_id(j).

### c- Trouver les utilisateurs les plus proches

In [12]:
def get_random_symm_matrix(size=4):
    similarity = np.zeros((size,size))
    for i in range(size):
        for j in range(size):
            if i==j:
                similarity[i,j] = 0
            else: 
                similarity[i,j] = np.random.rand()
                similarity[j,i] = similarity[i,j]
    return similarity

In [13]:
similarity = get_random_symm_matrix(10)

In [14]:
print(similarity)

[[ 0.          0.49818077  0.25763072  0.39301777  0.8052992   0.06085522
   0.42298104  0.89364427  0.27978438  0.11747913]
 [ 0.49818077  0.          0.70803795  0.12693327  0.82916492  0.39758701
   0.56686821  0.10121018  0.5204363   0.54685897]
 [ 0.25763072  0.70803795  0.          0.9306011   0.68520396  0.91664292
   0.25617291  0.39568905  0.58932435  0.18085064]
 [ 0.39301777  0.12693327  0.9306011   0.          0.13623248  0.38333467
   0.90289772  0.69267482  0.14509173  0.29646569]
 [ 0.8052992   0.82916492  0.68520396  0.13623248  0.          0.54794499
   0.84933069  0.329232    0.75390829  0.13487226]
 [ 0.06085522  0.39758701  0.91664292  0.38333467  0.54794499  0.
   0.85781849  0.16921563  0.96466229  0.53524898]
 [ 0.42298104  0.56686821  0.25617291  0.90289772  0.84933069  0.85781849
   0.          0.72770393  0.74541567  0.02405212]
 [ 0.89364427  0.10121018  0.39568905  0.69267482  0.329232    0.16921563
   0.72770393  0.          0.13046972  0.35963674]
 [ 0.279

In [15]:
# stores it as a dataframe (ie nearly bdd)
def get_bdd(matrix):
    bdd = pd.DataFrame(columns=['index1', 'index2', 'value'])
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i <= j:
                df2 = pd.DataFrame([[i,j,matrix[i,j]]], columns=['index1', 'index2', 'value'] )
                bdd=bdd.append(df2)
    bdd.index1 = bdd.index1.astype(int)
    bdd.index2 = bdd.index2.astype(int)
    return bdd

In [16]:
bdd = get_bdd(similarity)

In [17]:
def getSimilarUsers(user, bdd, nb_similar=3):
    sub_data = bdd.loc[(bdd.index1==user) | (bdd.index2==user)]
    sub_data = bdd.loc[((bdd.index1==user) | (bdd.index2==user)) & (bdd.index1!=bdd.index2)]
    other_index = np.zeros(len(sub_data.index))
    simil = np.zeros(len(sub_data.index))
    i=0
    for index, row in sub_data.iterrows():
        if row.index1 != user:
            other_index[i] = row.index1
        else:
            other_index[i] = row.index2
        simil[i] = row.value
        i+=1
    return other_index[np.argsort(-simil)[:nb_similar]].astype(int)

In [18]:
similar_1 = getSimilarUsers(1, bdd)
print(similar_1)

[4 2 6]


In [19]:
# check: we extract the line corresponding to the user 1 from the matrix:
similarity[1,:]

array([ 0.49818077,  0.        ,  0.70803795,  0.12693327,  0.82916492,
        0.39758701,  0.56686821,  0.10121018,  0.5204363 ,  0.54685897])

In [20]:
similarity[1,similar_1]

array([ 0.82916492,  0.70803795,  0.56686821])

Which are indeed the three highest.

### Apply to a fairly high number of users

In [21]:
similarity = get_random_symm_matrix(100)
bdd = get_bdd(similarity)

In [22]:
similar_89 = getSimilarUsers(89, bdd, nb_similar=20)

In [23]:
print(similar_89)

[56 12 63 33 66  1 99 60 84 96 73 22 43 57 10 46 25 20 26  9]
